In [ ]:
import uproot
import awkward as ak

import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt

from scipy.optimize import curve_fit

scale = 1.4

plt.rc("pdf", fonttype=42)
plt.rc("pdf", fonttype=42)
plt.rc("axes", labelsize=scale*20)
plt.rc("xtick", labelsize=scale*16, top=True, direction="in")
plt.rc("ytick", labelsize=scale*16, right=True, direction="in")
plt.rc("axes", titlesize=scale*20)
plt.rc("legend", fontsize=16, title_fontsize=16)

import os
curdir = os.getcwd()
while not os.path.exists(".projectroot") and os.getcwd() != os.path.expanduser('~'):
    os.chdir("..")
try:
    proot = open(".projectroot").readline()
except:
    print("Could not find file '.projectroot'.")
    print("Fallback: Look for the file 'rpad.py' in the current working directory (%s)." % curdir)
    print("Fonts in figures might not appear as intended, either.")
os.chdir(curdir)
import sys
import matplotlib.font_manager as font_manager
if len(proot) > 0:
    sys.path.append(proot+"/python")
    font_dir = proot + '/fonts'
    font_files = font_manager.findSystemFonts(font_dir)
    for f in font_files:
        font_manager.FontManager.addfont(font_manager.fontManager, path=f)
    plt.rcParams.update({"font.family": "serif", "font.serif": "Linux Libertine O", "font.cursive": "Linux Libertine O", "font.sans-serif": "Linux Libertine O", "mathtext.fontset": "custom"})
from rpad import rpad

os.makedirs("tmp", exist_ok=True)

dssds = ["U3", "U4", "U5"]
ids = [1, 2, 3]

In [ ]:
def f(v, a, b):
    return a*(1 + b*(1 - np.cos(v))/np.cos(v))

V = np.linspace(0, np.pi/4, 200)

In [ ]:
t, v, Emax = np.loadtxt("thickness_from_punchthrough_U3.dat", unpack=True)
Emax /= 1e3

In [ ]:
ts = np.unique(t)
ps = []
for x in ts:
    res = curve_fit(f, v[t == x], Emax[t == x], p0=(np.min(Emax[t == x]), 1.))
    ps.append(res[0])
print(ts)

In [ ]:
file = uproot.open(proot+"/data/bananas/all.root")
tree = file["a"]
b1 = tree.arrays(["fEdep", "angle"], "id == 1")
# b2 = tree.arrays(["Edep", "angle"], "id == 1")
x1, y1 = rpad(b1.angle, b1.fEdep)
# x2, y2 = rpad(b2.angle, b2.Edep)

# angle = np.append(x1, x2)
# angle = np.rad2deg(angle)
angle = np.rad2deg(x1)
# Edep  = np.append(y1, y2)
# Edep /= 1e3
Edep  = y1/1e3

# Initially wanted to plot ALL deposited energies in Delta-E-detector, including the ones without a hit in the 
# corresponding E-detector, but for illustrative purposes, it is better not to include these

In [ ]:
#plt.figure(figsize=(14,8))
plt.figure(figsize=(scale*6.4, scale*4.8))
plt.hist2d(angle, Edep, bins=(np.arange(-5, 50, 1), np.arange(0, 3., 0.01)), 
           cmap=plt.cm.Greys, norm=mpl.colors.LogNorm())
cbar = plt.colorbar()
cbar.ax.set_ylabel('Counts / (1 deg. $\\times$ 10 keV)')
plt.plot(np.rad2deg(V), f(V, *ps[6]), 'k--', label='67 µm')
#plt.plot(np.rad2deg(V), f(V, *ps[5]), 'k-.', label='65 µm')
plt.plot(np.rad2deg(V), f(V, *ps[3]), 'k-.', label='60 µm')
# for i in range(len(ps)):
#     plt.plot(np.rad2deg(V), f(V, *ps[i]), 'k--')
plt.xlim(-10, 50)
plt.ylim(0, 3.2)
plt.gca().set_yticks(np.arange(0.1, 3.2, 0.1), minor=True)
plt.gca().set_xticks(np.arange(0, 50, 10))
#plt.grid(which='both')
plt.annotate("$\Delta E$-contained\nevents", xy=(41, 1.930), xycoords='data', 
             xytext=(46, 2.250), textcoords='data', size=14,
             arrowprops=dict(arrowstyle="->", connectionstyle="angle,angleA=-92,angleB=180,rad=5", lw=1.2, mutation_scale=14), 
             horizontalalignment='center', verticalalignment='center', 
             bbox=dict(boxstyle='round', fc='white'))
plt.annotate("$\Delta E$-contained\nevents", xy=(41, 1.770), xycoords='data', 
             xytext=(46, 2.250), textcoords='data', size=14,
             arrowprops=dict(arrowstyle="->", connectionstyle="angle,angleA=-88,angleB=180,rad=5", lw=1.2, mutation_scale=14), 
             horizontalalignment='center', verticalalignment='center', 
             bbox=dict(boxstyle='round', fc='white', alpha=0.), alpha=0.)
plt.annotate("Punch\nthrough events", xy=(1, 0.850), xycoords='data', 
             xytext=(-2.5, 0.500), textcoords='data', size=14,
             arrowprops=dict(arrowstyle="->", connectionstyle="angle,angleA=-92,angleB=180,rad=5", lw=1.2, mutation_scale=14), 
             horizontalalignment='center', verticalalignment='center', 
             bbox=dict(boxstyle='round', fc='white'))
plt.annotate("Punch\nthrough events", xy=(1, 1.050), xycoords='data', 
             xytext=(-2.5, 0.500), textcoords='data', size=14,
             arrowprops=dict(arrowstyle="->", connectionstyle="angle,angleA=-88,angleB=180,rad=5", lw=1.2, mutation_scale=14), 
             horizontalalignment='center', verticalalignment='center', 
             bbox=dict(boxstyle='round', fc='white', alpha=0.), alpha=0.)
plt.annotate("Beta\nbackground", xy=(38, 0.300), xycoords='data', 
             xytext=(47, 0.650), textcoords='data', size=14,
             arrowprops=dict(arrowstyle="->", lw=1.2, mutation_scale=14), 
             horizontalalignment='center', verticalalignment='center', 
             bbox=dict(boxstyle='round', fc='white'))
plt.xlabel('$\\theta$ (deg.)')
plt.ylabel('$E_{\mathrm{dep}}$ (MeV)')
plt.text(-6.7, 2.8, "Punch through thresholds\nfor $\Delta E$ thicknesses:", fontsize=16, ha='left', va='center')
plt.text(-0.5, np.min(f(V, *ps[6])), "67 µm", fontsize=16, ha='right', va='center')
plt.text(-0.5, np.min(f(V, *ps[3])), "60 µm", fontsize=16, ha='right', va='center')
#plt.legend(title='Punch through thresholds\nfor $\Delta E$ thicknesses:', loc='upper left', alignment='left', framealpha=1)
plt.savefig('Edep_vs_angle.pdf', bbox_inches='tight')
plt.show()

In [ ]:
file.close()